<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe61622](https://jckantor.github.io/cbe61622);
content is available [on Github](https://github.com/jckantor/cbe61622.git).*


<!--NAVIGATION-->
< [C.8 Stepper Motors](https://jckantor.github.io/cbe61622/C.08-Stepper-Motors.html) | [Contents](toc.html) | [C.10 Wifi](https://jckantor.github.io/cbe61622/C.10-Wifi-with-ESP-01.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/C.09-PIO-Programming.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe61622/C.09-PIO-Programming.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# C.9 PIO Programming

Resources:

* [RP2040 Datasheet Section 3.4](https://datasheets.raspberrypi.com/rp2040/rp2040-datasheet.pdf)

Life with David

## C.9.1 Setting multiple pins from Python

In [5]:
%serialconnect

from machine import Pin
import time
from rp2 import PIO, StateMachine, asm_pio

# decorator to translate to PIO machine code
@asm_pio(
    out_init = (rp2.PIO.OUT_LOW,) * 8,     # initialize 8 consecutive pins
    out_shiftdir = rp2.PIO.SHIFT_RIGHT)    # output lsb bits first

def parallel_prog():
    pull(block)                             # pull data from Tx FIFO. Wait for data
    out(pins, 8)                            # send 8 bits from OSR to pins

# create an instance of the state machine
sm = StateMachine(0, parallel_prog, freq=1000000, out_base=Pin(0))

# start the state machine
sm.active(1)

for n in range(256):
    sm.put(n)
    time.sleep(0.01)


serial exception on close write failed: [Errno 6] Device not configured
Found serial ports: /dev/cu.usbmodem14101, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14101 --baud=115200 
Ready.


## C.9.2 Writing Stepper Steps to pins


In [39]:
%serialconnect

from machine import Pin
import time
from rp2 import PIO, StateMachine, asm_pio

# decorator to translate to PIO machine code
@asm_pio(
    out_init = (rp2.PIO.OUT_LOW,) * 4,     # initialize 8 consecutive pins
    out_shiftdir = rp2.PIO.SHIFT_RIGHT)    # output lsb bits first

def stepper_step():
    pull(block)                             # pull data from Tx FIFO. Wait for data
    out(pins, 4)                            # send 8 bits from OSR to pins

# create an instance of the state machine
sm = StateMachine(0, stepper_step, freq=1000000, out_base=Pin(0))

# start the state machine
sm.active(1)

step_sequence = [8, 12, 4, 6, 2, 3, 1, 9]
 
for n in range(500):
    sm.put(step_sequence[n % len(step_sequence)])
    time.sleep(0.01)

Found serial ports: /dev/cu.usbmodem14101, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14101 --baud=115200 
Ready.
.

In [94]:
%serialconnect

from machine import Pin
import time
from rp2 import PIO, StateMachine, asm_pio

# decorator to translate to PIO machine code
@asm_pio(
    set_init = rp2.PIO.OUT_LOW)

def count_blink():
    pull()
    mov(x, osr)
    label("count")
    set(pins, 1)
    set(y, 100)
    label("on")
    nop()        [1]
    jmp(y_dec, "on")
    set(pins, 0)
    nop()        [19]
    nop()        [19]
    nop()        [19]
    nop()        [19]
    jmp(x_dec, "count")

# create an instance of the state machine
sm = StateMachine(0, count_blink, freq=2000, set_base=Pin(25))

# start the state machine
sm.active(1)
sm.put(20)


Found serial ports: /dev/cu.usbmodem14101, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14101 --baud=115200 
Ready.


In [92]:
%serialconnect

from machine import Pin
import time
from rp2 import PIO, StateMachine, asm_pio

# decorator to translate to PIO machine code
@asm_pio(
    out_init = (rp2.PIO.OUT_LOW,) * 4,     # initialize 8 consecutive pins
    out_shiftdir = rp2.PIO.SHIFT_RIGHT)    # output lsb bits first

def stepper_step():
    pull(block)                             # pull data from Tx FIFO. Wait for data
    out(pins, 4)                            # send 8 bits from OSR to pins

# create an instance of the state machine
sm = StateMachine(0, stepper_step, freq=1000000, out_base=Pin(0))

# start the state machine
sm.active(1)

step_sequence = [8, 12, 4, 6, 2, 3, 1, 9]

def step():
    pos = 0
    while True:
        coils = step_sequence[pos % len(step_sequence)]
        yield coils
        pos += 1
        
stepper = step()
for k in range(10):
    c = next(stepper)
    print(c)
    


#for n in range(100):
#    for step in step_sequence:
#        sm.put(step)
#        time.sleep(0.01)

serial exception on close write failed: [Errno 6] Device not configured
Found serial ports: /dev/cu.usbmodem14101, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14101 --baud=115200 
Ready.
8
12
4
6
2
3
1
9
8
12


## C.9.3 Interacting PIO programs

Section 3.2.7 of the RP2040 data sheet describes how interactions between state machines on the same PIO processor can be managed. Here will demonstrate this in a few steps. For the first step, we create a counter that accepts an integer from the FIFO buffer, then blinks an led a fixed number of times.

In [28]:
%serialconnect

from machine import Pin
import time
import rp2
from rp2 import PIO, StateMachine, asm_pio

@asm_pio(out_init = rp2.PIO.OUT_LOW)
def count_blink():
    pull(block)            # wait for data on Tx FIFO
    set(pins, 1)
    set(x, osr)
    
# create an instance of the state machine
sm0 = StateMachine(0, count_blink, freq=2000, out_base=Pin(25))

# start the state machine
sm0.active(1)
sm0.put(1)
time.sleep(2)
sm0.active(0)

serial exception on close write failed: [Errno 6] Device not configured
Found serial ports: /dev/cu.usbmodem14101, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14101 --baud=115200 
Ready.


In [37]:
%serialconnect

from machine import Pin
import time
from rp2 import PIO, StateMachine, asm_pio

# decorator to translate to PIO machine code
@asm_pio(
    out_init = (rp2.PIO.OUT_LOW,) * 4,     # initialize 8 consecutive pins
    out_shiftdir = rp2.PIO.SHIFT_RIGHT)    # output lsb bits first

def stepper_step():
    pull(block)                             # pull data from Tx FIFO. Wait for data
    out(pins, 4)                            # send 8 bits from OSR to pins

# create an instance of the state machine
sm = StateMachine(0, stepper_step, freq=1000000, out_base=Pin(0))

# start the state machine
sm.active(1)

step_sequence = [8, 12, 4, 6, 2, 3, 1, 9]
 
for n in range(1000):
    sm.put(step_sequence[n % len(step_sequence)])
    time.sleep(0.01)
        

Found serial ports: /dev/cu.usbmodem14101, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14101 --baud=115200 
Ready.
..

<!--NAVIGATION-->
< [C.8 Stepper Motors](https://jckantor.github.io/cbe61622/C.08-Stepper-Motors.html) | [Contents](toc.html) | [C.10 Wifi](https://jckantor.github.io/cbe61622/C.10-Wifi-with-ESP-01.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/C.09-PIO-Programming.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe61622/C.09-PIO-Programming.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>